# Setting up the scraping environment

In [34]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import requests
import re
from tqdm import tqdm
import time

# Looping through the pages to gather article links

In [3]:
# Giving label names that can be used in the URL link
labels = ['belfold', 'kulfold', 'gazdasag', 'video', 'eletmod', 'sport', 'techtud', 'after', 'english']
pages = 5
labels = ['belfold']
observations = (pages - 1) * len(labels) * 10
list_pattern = [None] * observations

links = list_pattern
counter = 0
for j in tqdm(labels):
    for i in range(1,pages):
        # Giving current URL link
        url = 'https://telex.hu/rovat/' + j + '?oldal=' + str(i)
        
        # Constructing the soup based on the current URL
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Accessing the article links with CSS selector
        elements = soup.select('a.list__item__title')
        
        # Saving link parts into a list
        pattern = r'href="([^"]+)"'
        for k in range(10):
            text = str(elements[k])
            link = re.findall(pattern, text)
            link = 'https://telex.hu' + link[0]
            links[counter] = link
            counter += 1
            
#links

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.31s/it]


In [8]:
len(links)
links[-1]

'a'

# Accessing links

### Collencting authors

In [140]:
author_primary = [None] * observations
author_secondary = [None] * observations
author_tertary = [None] * observations
author_quaternary = [None] * observations
counter = 0

for i in tqdm(links):
    url = i
    
    # Constructing the soup based on the current URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Accessing the article links with CSS selector
    elements = soup.select('a.author__name')
    
    # Extracting authors into a list
    authors_temp = []
    for j in range(len(elements)):
        
        text_temp = str(elements[j])
        first_newline_index = text_temp.find('\n')
        second_newline_index = text_temp.find('\n', first_newline_index + 1)

        author = text_temp[first_newline_index + 1:second_newline_index]
        author = author.strip()
        
        authors_temp.append(author)

    # Adding authors to the corresponding lists
    if len(authors_temp) == 1:
        author_primary[counter] = authors_temp[0]
        counter += 1
        
    if len(authors_temp) == 2:
        author_primary[counter] = authors_temp[0]
        author_secondary[counter] = authors_temp[1]
        counter += 1
        
    if len(authors_temp) == 3:
        author_primary[counter] = authors_temp[0]
        author_secondary[counter] = authors_temp[1]
        author_tertary[counter] = authors_temp[2]
        counter += 1
        
    if len(authors_temp) >= 4:
        author_primary[counter] = authors_temp[0]
        author_secondary[counter] = authors_temp[1]
        author_tertary[counter] = authors_temp[2]
        author_quaternary[counter] = authors_temp[3]
        counter += 1
    
#author_secondary

100%|██████████████████████████████████████████████████████████████████████████████| 2990/2990 [11:48<00:00,  4.22it/s]


In [141]:
len(author_primary)
#author_primary

2990

### Collecting title

In [142]:
titles = [None] * observations
counter = 0

for i in tqdm(links):
    url = i
    
    # Constructing the soup based on the current URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Accessing the article links with CSS selector
    elements = soup.select('h1')
    
    # Extracting title
    title = str(elements[0])[5:-5]
    title = title.strip()
    titles[counter] = title
    counter += 1
    
#titles

100%|██████████████████████████████████████████████████████████████████████████████| 2990/2990 [04:57<00:00, 10.04it/s]


### Collecting tags

In [143]:
tag_primary = [None] * observations
tag_secondary = [None] * observations
tag_tertary = [None] * observations
tag_quaternary = [None] * observations
counter = 0

for i in tqdm(links):
    url = i
    
    # Constructing the soup based on the current URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Accessing the article links with CSS selector
    elements = soup.select('div.title-section__tags.content-wrapper__child')
    
    # Extracting tags into a list
    tags_temp = []
    for j in range(len(elements)):
        
        text_temp = str(elements[j])
        first_newline_index = text_temp.find('\n')
        second_newline_index = text_temp.find('\n', first_newline_index + 1)

        tag = text_temp[first_newline_index + 1:second_newline_index]
        tag = tag.strip()
        
        tags_temp.append(tag)
        
    # Adding tags to the corresponding lists
    if len(tags_temp) == 1:
        tag_primary[counter] = tags_temp[0]
        counter += 1
        
    if len(tags_temp) == 2:
        tag_primary[counter] = tags_temp[0]
        tag_secondary[counter] = tags_temp[1]
        counter += 1
        
    if len(tags_temp) == 3:
        tag_primary[counter] = tags_temp[0]
        tag_secondary[counter] = tags_temp[1]
        tag_tertary[counter] = tags_temp[2]
        counter += 1
        
    if len(tags_temp) >= 4:
        tag_primary[counter] = tags_temp[0]
        tag_secondary[counter] = tags_temp[1]
        tag_tertary[counter] = tags_temp[2]
        tag_quaternary[counter] = tags_temp[3]
        counter += 1
    
#tag_secondary

100%|██████████████████████████████████████████████████████████████████████████████| 2990/2990 [05:01<00:00,  9.93it/s]


### Collecting the text

In [ ]:
texts = [None] * observations
counter = 0

for i in tqdm(links):
    url = i
    
    # Constructing the soup based on the current URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Accessing the article links with CSS selector
    elements = soup.select('p')
    
    # Extracting text into a string
    text = ''
    if len(str(elements[4])[3:-4].strip()) < 10:
        elements = elements[1:]
    for j in range(4,len(elements)):
        if str(elements[j])[3] != '<':
            text = text + ' ' + str(elements[j])[3:-5]
            
    # Deleting longer links from the text
    pattern = r'<a\s+href="[^"]*"\s+target="_blank">'
    text = re.sub(pattern, '', text)
    
    pattern = r'<a\s+href="[^"]*"[^>]*>'
    text = re.sub(pattern, '', text)

    # Deleting additional shit from the text
    substring_to_delete = '</a>'
    text = text.replace(substring_to_delete, '')
    substring_to_delete = '<i>'
    text = text.replace(substring_to_delete, '')
    substring_to_delete = '</i>'
    text = text.replace(substring_to_delete, '')
    substring_to_delete = '<b>'
    text = text.replace(substring_to_delete, '')
    substring_to_delete = '</b>'
    text = text.replace(substring_to_delete, '')
    substring_to_delete = 'class="article-html-content article__lead">\n'
    text = text.replace(substring_to_delete, '')
    
    text = text.strip()
    texts[counter] = text
    counter += 1
    
#texts

# Creating the dataframe

In [ ]:
articles = pd.DataFrame({'link': links,
                         'titles': titles,
                         'author_primary': author_primary,
                         'author_secondary': author_secondary,
                         'author_tertary': author_tertary,
                         'author_quaternary': author_quaternary,
                         'tag_primary': tag_primary,
                         'tag_secondary': tag_secondary,
                         'tag_tertary': tag_tertary,
                         'tag_quaternary': tag_quaternary,
                         'texts': texts})

In [ ]:
articles[:-10]

In [ ]:
articles.to_csv('articles.csv', encoding='utf-8', index=False)

# Creating the whole scraping code in a single loop

In [ ]:
# Giving label names that can be used in the URL link
labels = ['belfold', 'kulfold', 'gazdasag', 'video', 'eletmod', 'sport', 'techtud', 'after', 'english']
labels = ['belfold']
#observations = (pages - 1) * len(labels) * 10
#list_pattern = [None] * observations
year_substring = '/2022/'

links = ['Imi']

for l in labels:
    page = 0
    while year_substring not in links[-1]:
        # Downloading links
        titles = []
        if len(links) == 1:
            links = []
        for i in range(1,51):
            page += 1
            # Giving current URL link
            url = 'https://telex.hu/rovat/' + l + '?oldal=' + str(page)
            
            # Constructing the soup based on the current URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Accessing the article links with CSS selector
            elements = soup.select('a.list__item__title')
            
            # Saving link parts into a list
            pattern = r'href="([^"]+)"'
            for k in range(10):
                text = str(elements[k])
                link = re.findall(pattern, text)
                link = 'https://telex.hu' + link[0]
                links.append(link)
        # Accessing links
        # Collecting authors
        author_primary = []
        author_secondary = []
        author_tertary = []
        author_quaternary = []
        
        for i in links:
            url = i

            # Constructing the soup based on the current URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Accessing the article links with CSS selector
            elements = soup.select('a.author__name')

            # Extracting authors into a list
            authors_temp = []
            
            for j in range(len(elements)):

                text_temp = str(elements[j])
                first_newline_index = text_temp.find('\n')
                second_newline_index = text_temp.find('\n', first_newline_index + 1)

                author = text_temp[first_newline_index + 1:second_newline_index]
                author = author.strip()

                authors_temp.append(author)

            # Adding authors to the corresponding lists
            if len(authors_temp) == 1:
                author_primary.append(authors_temp[0])
                author_secondary.append(None)
                author_tertary.append(None)
                author_quaternary.append(None)

            if len(authors_temp) == 2:
                author_primary.append(authors_temp[0])
                author_secondary.append(authors_temp[1])
                author_tertary.append(None)
                author_quaternary.append(None)

            if len(authors_temp) == 3:
                author_primary.append(authors_temp[0])
                author_secondary.append(authors_temp[1])
                author_tertary.append(authors_temp[2])
                author_quaternary.append(None)

            if len(authors_temp) >= 4:
                author_primary.append(authors_temp[0])
                author_secondary.append(authors_temp[1])
                author_tertary.append(authors_temp[2])
                author_quaternary.append(authors_temp[3])
            
        # Collecting titles
        titles = []
            
        for i in links:
            url = i

            # Constructing the soup based on the current URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Accessing the article links with CSS selector
            elements = soup.select('h1')

            # Extracting title
            title = str(elements[0])[5:-5]
            title = title.strip()
            titles.append(title)
    
        # Collecting tags
        tag_primary = []
        tag_secondary = []
        tag_tertary = []
        tag_quaternary = []

        for i in links:
            url = i

            # Constructing the soup based on the current URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Accessing the article links with CSS selector
            elements = soup.select('div.title-section__tags.content-wrapper__child')

            # Extracting tags into a list
            tags_temp = []
            for j in range(len(elements)):

                text_temp = str(elements[j])
                first_newline_index = text_temp.find('\n')
                second_newline_index = text_temp.find('\n', first_newline_index + 1)

                tag = text_temp[first_newline_index + 1:second_newline_index]
                tag = tag.strip()

                tags_temp.append(tag)

            # Adding tags to the corresponding lists
            if len(tags_temp) == 1:
                tag_primary.append(tags_temp[0])
                tag_secondary.append(None)
                tag_tertary.append(None)
                tag_quaternary.append(None)
                
            if len(tags_temp) == 2:
                tag_primary.append(tags_temp[0])
                tag_secondary.append(tags_temp[1])
                tag_tertary.append(None)
                tag_quaternary.append(None)

            if len(tags_temp) == 3:
                tag_primary.append(tags_temp[0])
                tag_secondary.append(tags_temp[1])
                tag_tertary.append(tags_temp[2])
                tag_quaternary.append(None)

            if len(tags_temp) >= 4:
                tag_primary.append(tags_temp[0])
                tag_secondary.append(tags_temp[1])
                tag_tertary.append(tags_temp[2])
                tag_quaternary.append(tags_temp[3])
                
        # Collecting texts
        texts = []
        
        for i in links:
            url = i

            # Constructing the soup based on the current URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Accessing the article links with CSS selector
            elements = soup.select('p')

            # Extracting text into a string
            text = ''
            if len(str(elements[4])[3:-4].strip()) < 10:
                elements = elements[1:]
            for j in range(4,len(elements)):
                if str(elements[j])[3] != '<':
                    text = text + ' ' + str(elements[j])[3:-5]

            # Deleting longer links from the text
            pattern = r'<a\s+href="[^"]*"\s+target="_blank">'
            text = re.sub(pattern, '', text)

            pattern = r'<a\s+href="[^"]*"[^>]*>'
            text = re.sub(pattern, '', text)

            # Deleting additional shit from the text
            substring_to_delete = '</a>'
            text = text.replace(substring_to_delete, '')
            substring_to_delete = '<i>'
            text = text.replace(substring_to_delete, '')
            substring_to_delete = '</i>'
            text = text.replace(substring_to_delete, '')
            substring_to_delete = '<b>'
            text = text.replace(substring_to_delete, '')
            substring_to_delete = '</b>'
            text = text.replace(substring_to_delete, '')
            substring_to_delete = 'class="article-html-content article__lead">\n'
            text = text.replace(substring_to_delete, '')

            text = text.strip()
            texts.append(text)
            
        # Writing dataframe
        articles = pd.DataFrame({'link': links,
                         'titles': titles,
                         'author_primary': author_primary,
                         'author_secondary': author_secondary,
                         'author_tertary': author_tertary,
                         'author_quaternary': author_quaternary,
                         'tag_primary': tag_primary,
                         'tag_secondary': tag_secondary,
                         'tag_tertary': tag_tertary,
                         'tag_quaternary': tag_quaternary,
                         'texts': texts})
        
        # Saving the dataframe
        end_page = str(page)
        start_page = str(page - 49)
        
        folder_path = 'C:/Users/imre2/Desktop/'
        saving_name = os.path.join(folder_path, f"telex_{l}_{start_page}_{end_page}.csv")
        
        articles.to_csv(saving_name, index=False)
        
        print(l, page)